In [42]:
from random import randint

M = 1000

lst = [randint(1, 1001) for _ in range(M)]

In [33]:
def first_approach_function(lst):
    sorted_lst = sorted(lst)
    counter = 0
    max_counter = 0
    prev_elem = lst[0]
    for elem in sorted_lst[1:]:
        if elem != prev_elem:
            if counter > max_counter:
                max_counter = counter
                most_frequent_elem = prev_elem
            counter = 0
        counter += 1
        prev_elem = elem
    return most_frequent_elem

In [25]:
def second_approach_function(lst):
    frequency_dict = {}
    for elem in lst:
        corresponding_counter = frequency_dict.get(elem)
        if corresponding_counter is None:
            frequency_dict[elem] = 1
        else:
            frequency_dict[elem] += 1
    return max(frequency_dict, key=lambda key : frequency_dict[key])

In [ ]:
def third_approach_function(lst):
    """
    То же самое, что и через словарь, но
    вместо ключей используются индексы в массиве,
    т.е. элемент 49 - значит прибавляем единицу к
    сорок девятой ячейке, которая была до этого нулем
    (вспомогательный массив инициализируется нулевым)
    """
    auxillary_length = max(lst)
    counter_lst = [0] * auxillary_length
    for i in lst:
        counter_lst[i] += 1
    most_frequent_el = max(counter_lst)
    return counter_lst.index(most_frequent_el)

In [1]:
import numpy as np

def fourth_approach_function(lst):
    np_lst = np.array(lst)
    frequency_lst = np.bincount(np_lst)
    return (np.where(frequency_lst == max(frequency_lst))[0][0])